In [28]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_checkpoint = "Yukang/Llama-2-7b-longlora-32k-ft"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint, device_map="auto", low_cpu_mem_usage=True, torch_dtype=torch.float16
)

In [29]:
if tokenizer.add_bos_token:
    tokenizer.add_bos_token = False

In [4]:
from datasets import load_dataset

dataset = load_dataset("w11wo/FourSquare-NYC-POI")

In [17]:
dataset = dataset.map(lambda x: tokenizer(x["llama_prompt"]), batched=False)

Map: 100%|██████████| 1429/1429 [00:02<00:00, 687.29 examples/s]


In [5]:
test_str = """<s>[INST] <<SYS>>
You are user 902 and your basic information is as follows:
Age: adult; Gender: female; Education: college & beyond; SocioEco: middle.
You have the following traits: extroverted, agreeable, conscientious, emotionally stable, open.
You have the following preferences: socializing, exploring new cafés, enjoying sweets, visiting bars, traveling via train.
You have the following routines: frequent visits to cafés, shopping at department stores, regular trips to train stations, working in an office, indulging in treats from cupcake shops.
User 902 is a sociable and friendly adult who enjoys exploring the urban environment around her. Frequently visiting cafés and cupcake shops showcases her love for social gatherings and sweet delights. She has a solid educational background, likely holding a college degree, which aligns with her conscientious nature in managing her life and work responsibilities from her regular visits to the office. User 902 exhibits openness with a flair for experiencing diverse flavors, seen in her trips to food trucks and bars. She is emotionally stable and likely has a close-knit group of friends with whom she enjoys outings, indicated by her frequent visits to lively spots. Train stations appear often in her check-ins suggesting she enjoys traveling and exploring different areas. Future potential places of interest may include new cafés, food markets, and cultural festivals in the city, reflecting her adventurous and social spirit.
<</SYS>>

The following data is a trajectory of user 902: At 2012-04-25 09:30:56, user 902 visited POI id 1090 which is a Cupcake Shop and has Category id 188. At 2012-04-25 13:58:55, user 902 visited POI id 950 which is a Food Truck and has Category id 202.
Given the data, At 2012-04-25 21:24:11, Which POI id will user 902 visit? Note that POI id is an integer in the range from 0 to 4981. [/INST] At 2012-04-25 21:24:11, user 902 will visit POI id 2955. </s>"""

In [30]:
def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)))

In [31]:
print_tokens_with_ids("4981. [/INST] At")

[('▁', 29871), ('4', 29946), ('9', 29929), ('8', 29947), ('1', 29896), ('.', 29889), ('▁[', 518), ('/', 29914), ('INST', 25580), (']', 29962), ('▁At', 2180)]


In [32]:
print_tokens_with_ids(" [/INST]")

[('▁[', 518), ('/', 29914), ('INST', 25580), (']', 29962)]


In [18]:
from tqdm.auto import tqdm

max(len(dataset["train"][i]["input_ids"]) for i in tqdm(range(len(dataset["train"]))))

100%|██████████| 11022/11022 [00:07<00:00, 1401.74it/s]


13904

In [19]:
max(len(dataset["test"][i]["input_ids"]) for i in tqdm(range(len(dataset["test"]))))

100%|██████████| 1429/1429 [00:00<00:00, 1651.39it/s]


10532